# TensorFlow: DNN using transfer learning

In [ ]:
import os
import random
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import tensorflow as tf

print("TF Version: ", tf.__version__)
print("TF Eager mode: ", tf.executing_eagerly())
print("TF GPU is", "available" if tf.config.list_physical_devices("GPU") else "not available")

In [ ]:
from common import CV_DATA_DIR
DATA_DIR = CV_DATA_DIR / "animals" / "cats-and-dogs"
assert DATA_DIR.is_dir(),  \
    f"Dir <{DATA_DIR}> does not exists"

In [ ]:
nrows = 4
ncols = 4

cats_names = os.listdir(DATA_DIR / "Cat")
dogs_names = os.listdir(DATA_DIR / "Dog")

print("Total number of cat pictures: ", len(cats_names))
print("Total number of dog pictures: ", len(dogs_names))

fig = plt.gcf()
fig.set_size_inches((ncols * 3, nrows * 3))

next_cat_pix = [DATA_DIR / "Cat" / fname  for fname in random.sample(cats_names, k=8)]
next_dog_pix = [DATA_DIR / "Dog" / fname  for fname in random.sample(dogs_names, k=8)]

for i, img_path in enumerate(next_cat_pix + next_dog_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis("off")
    image = mpimg.imread(img_path)
    plt.imshow(image)

# Prepare dataset

In [12]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=DATA_DIR,
    image_size=(150,150),
    batch_size=64,
    label_mode="binary",
    validation_split=0.1,
    subset="both",
    seed=1)

Found 22434 files belonging to 2 classes.
Using 20191 files for training.
Using 2243 files for validation.


In [13]:
train_ds = (train_ds
            .cache()
            .shuffle(buffer_size=1000)
            .prefetch(tf.data.AUTOTUNE))

val_ds = (val_ds
          .cache()
          .prefetch(tf.data.AUTOTUNE))

In [ ]:
# Pre-process input data using particular method
# See https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input
def preprocess(image, label):
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)

# Define model

## Define base model

* Get the architecture of existing model
* Load weights to existing model
* Freeze the weights of the layers
* Select last layer to chain with

In [ ]:
# Link to the file with weights of pre-trained model
URL = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

weights_file = tf.keras.utils.get_file(
    origin=URL
)

In [ ]:
# Declare model without top (Dense) layer
base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(150,150,3),
    include_top=False,
    weights=None
)

# Load weights
base_model.load_weights(weights_file)

# Freeze the base model
base_model.trainable = False

In [ ]:
# Print model summary
base_model.summary()

In [ ]:
# Choose `mixed7` as the last layer of base model
last_layer = base_model.get_layer("mixed7")
print(f"Last layer output shape: {last_layer.output.shape}")
last_output = last_layer.output

# Chain tuning layers

* Add layers for tuning (training) to the top of the existing model

In [ ]:
# Add layers to the pre-trained model (layers to learn weights)
last_output = tf.keras.layers.Flatten()(last_output)
last_output = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(last_output)
last_output = tf.keras.layers.Dropout(0.2)(last_output)
last_output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(last_output)

In [ ]:
# Create final model without augmentation
model_without_aug = tf.keras.Model(base_model.input, last_output)

In [ ]:
# Print model summary
model_without_aug.summary()

## Chain augmentation layers

In [ ]:
aug_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.4),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomContrast(0.4),
    tf.keras.layers.RandomZoom(0.2),
])

In [ ]:
inputs = tf.keras.Input(shape=(150,150,3))
last_output = aug_layers(inputs)
last_output = model_without_aug(last_output)

model_with_aug = tf.keras.Model(inputs, last_output)

In [ ]:
model_with_aug.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy'])

# Train model

In [ ]:
EPOCHS = 20

%%time
history = model_with_aug.fit(
    train_ds,
    validation_data = val_ds,
    epochs = EPOCHS,
    verbose = 2)

# Evaluate

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(acc))

fig, ax = plt.subplots(1,2, figsize=(12, 6))
ax[0].plot(epochs, acc, "bo", label="Training accuracy")
ax[0].plot(epochs, val_acc, "b", label="Validation accuracy")
ax[0].set_title("Training and validation accuracy")
ax[0].set_xlabel("epochs")
ax[0].set_ylabel("accuracy")
ax[0].legend()

ax[1].plot(epochs, loss, "bo", label="Training Loss")
ax[1].plot(epochs, val_loss, 'b', label="Validation Loss")
ax[1].set_title("Training and validation loss")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

plt.show()